<a href="https://colab.research.google.com/github/Shopping-Yuan/ML2021HW/blob/Shopping_vscode_branch/HW05_modified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

part 1 load file (finish)
part 2 clean data (finish on 12/5)
part 3 tokenize : sentencepiece (finish on 12/6)
part 4 build model encoder decoder (finish on 12/7)
part 5 beam search (finish on 12/8)
part 6 label smooth (finish on 12/9)

part 7 test (finish on 12/10)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import shutil
shutil.copyfile('/content/drive/MyDrive/ted2020.tgz','/content/ted2020.tgz')

Mounted at /content/drive


'/content/ted2020.tgz'

In [ ]:
import tarfile

rawdata_file = "/content/ted2020.tgz"
unzip_path = "/content/train_dev/"
# open file
file = tarfile.open(rawdata_file)

# extracting file
file.extractall(unzip_path)

file.close()


In [ ]:
src_lang = "raw.en"
tgt_lang = "raw.zh"

src_path = f"{unzip_path}{src_lang}"
tgt_path = f"{unzip_path}{tgt_lang}"
print(tgt_path)

/content/train_dev/raw.zh


In [ ]:
print(chr(10022).join([chr(21407), chr(31070), chr(21855), chr(21205)]))

原✦神✦啟✦動


In [ ]:
# for path in [src_path,tgt_path]:
#   with open(path, "r") as f:
  #  data = f.readlines()
  #  print(type(data),data[0:5])#<class 'list'> ['Thank you so much, Chris.\n', "And it'......on.\n']
    # data = f.read().splitlines()
    # print(type(data),data[0:5],len(data))

In [ ]:
# import re
# print(ord("，"))
# print(chr(65292))

In [ ]:
import unicodedata
def to_halfwidth(string):
  return "".join(unicodedata.normalize('NFKC',letter) for letter in string)
# print(to_halfwidth("ＡＳＤＦＦＦＦＱ")) #ASDFFFFQ

In [ ]:
import string
import re
def clean_s_zh(s):
    s = to_halfwidth(s)
    # step 1 : delete — _
    delete = " _()[]"
    delete_rules = s.maketrans("","",delete)
    s = s.translate(delete_rules)

    # step 2 : replace “” with ""
    to_be_replace = '“”'
    replace = '""'
    replace_dict = dict(zip(to_be_replace,replace))
    # print(replace_dict.items()) : dict_items([('“', '"'), ('”', '"')])

    # step 3 : add **END** before and after punctuation

    """
    The number of sentences in one line may be different
    in line pairs of source and target set.
    so I try to use "。!?" or ".!?" to split sentences.

    """

#    punctuation = "。,;!?()\"~「」"
    punctuation = "♪♫。!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    zh_list = s.strip("\n").split("\n")

    return zh_list

def clean_s_en(s):
    s = to_halfwidth(s)

    replace_dict = {}

    delete = "-()[]"
    for char in delete:
      replace_dict[char] = ""

    punctuation = "♪♫!?"
    for char in punctuation:
      replace_dict[char] = char + "**END**"
    replace_rules = s.maketrans(replace_dict)
    s = s.translate(replace_rules)

    pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
    s = pattern.sub("**END**",s)

    en_list = s.strip("\n").split("\n")

    return en_list

In [ ]:
# pattern = re.compile(r"(?<!([.\s\r\n\f][a-zA-Z]))[.]")
# result = pattern.sub("**END**","There are many people in U.S. w.r.t. in Taiwan.Thank you.")
# print(result)

In [ ]:
def load_file(path,fuction):
  with open(path, "r") as f:
    data = f.read()
    return(fuction(data))
src_list = load_file(src_path,clean_s_en)
tgt_list = load_file(tgt_path,clean_s_zh)

In [ ]:
# with open(tgt_path, "r") as f:
#     data = f.readlines()
#     print(data[17])
# print(tgt_list[17])

In [ ]:
def divide_by_END(s):
    list_s = []
    for line_string in s.strip("**END**").split("**END**"):
      if line_string not in [""," "]:
         list_s.append(line_string)
    return(list_s)

def devide_en_again(s,punctuation = ":;"):
    replace_dict = {}
    for char in punctuation:
      replace_dict[char] = char + "**END**"

    replace_rules_src = s.maketrans(replace_dict)
    new_s = divide_by_END(s.translate(replace_rules_src))
    return new_s
def remove_too_long(src_list,tgt_list,s_threshold = 400, t_threshold = 120):
    too_long_src = 0
    too_long_tgt = 0
    remove = False
    new_s = []
    new_t = []
    for i in range(len(src_list)):
      if ((len(src_list[i])>s_threshold)):
        remove = True
        too_long_src += 1
      if (len(tgt_list[i])>t_threshold):
        remove = True
        too_long_tgt += 1
      if remove == False:
        new_s.append(src_list[i])
        new_t.append(tgt_list[i])
      else :
        remove = False
    return(new_s,new_t,too_long_src,too_long_tgt)


In [ ]:
def check_data_pairs(src_list,tgt_list):
    index = 0
    new_src_list = []
    new_tgt_list = []

    same = 0
    add_next = 0
    split_again = 0
    not_use = 0

    while(index < len(src_list)):

      src = divide_by_END(src_list[index])
      tgt = divide_by_END(tgt_list[index])
      # case 1 : src is as long as tgt , finished.
      if len(src) == len(tgt):
        new_src_list += src
        new_tgt_list += tgt
        same += 1
        index += 1

      else :
        # if it is not the last one : both src and tgt add next sentence
        if index != len(src_list)-1:
          src_add_next = divide_by_END(src_list[index] + src_list[index+1])
          tgt_add_next = divide_by_END(tgt_list[index] + tgt_list[index+1])
          # case 2 : src_add_next is as long as tgt_add_next , finished.
          if len(src_add_next) == len(tgt_add_next):
            new_src_list += src_add_next
            new_tgt_list += tgt_add_next
            add_next += 2
            index += 2

          # using new punctuation to divide tgt (english) sentence.
          else :
            src_add_next = devide_en_again(src_list[index] + src_list[index+1])
            # case 3 : src_add_next is as long as tgt_add_next , finished.
            if len(src_add_next) == len(tgt_add_next):
              new_src_list += src_add_next
              new_tgt_list += tgt_add_next
              split_again +=2
              index += 2

            # case 4 : sentence will not be used.
            else :
              not_use += 1
              # if to_do == 1 :
              #   print(index,src_add_next,tgt_add_next,len(src_add_next),len(tgt_add_next))
              index += 1

        # if it is the last one
        else :
          not_use += 1
          index += 1
    print(len(new_src_list))
    new_src_list,new_tgt_list,too_long_src,too_long_tgt = remove_too_long(new_src_list,new_tgt_list)
    print(index,same,add_next,split_again,not_use,too_long_src,too_long_tgt,len(new_src_list))

    return(new_src_list,new_tgt_list)

In [ ]:
new_src_list,new_tgt_list = check_data_pairs(src_list,tgt_list)
# print(dataset[0][0],dataset[1][0])

389602
394066 350421 12284 2980 28381 677 605 388728


In [ ]:
data_path = "/content/train_dev/data.txt"
label_path = "/content/train_dev/label.txt"
with open(data_path, "w") as f:
  f.write("\n".join(new_src_list))
with open(label_path, "w") as f:
  f.write("\n".join(new_tgt_list))

In [ ]:
def get_max(l):
  exceed = 0
  maxize_sentence_length = 0
  for sentence in l:
    if len(sentence) > maxize_sentence_length:
      maxize_sentence_length = len(sentence)
  return(maxize_sentence_length)

data_info = {
    "data":{"path":data_path,"lang":"en","max_l":get_max(new_src_list)},
    "label":{"path":label_path,"lang":"zh","max_l":get_max(new_tgt_list)},
}
print(get_max(new_src_list),get_max(new_tgt_list))

400 120


In [ ]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.9 MB/s eta 0:00:00


In [ ]:
import sentencepiece as spm
vocab_size = 8000
def tokenized(data_or_label,data_info=data_info):
  spm.SentencePieceTrainer.train(
      input=data_info[data_or_label]["path"],
      model_prefix=f'spm{vocab_size}_{data_info[data_or_label]["lang"]}',
      vocab_size=vocab_size,
      character_coverage=1,
      model_type='unigram', # 'bpe' works as well
      input_sentence_size=400000,
      shuffle_input_sentence=True,
      normalization_rule_name='nmt_nfkc_cf',
      unk_id=1,
      bos_id=2,
      eos_id=3,
      pad_id=0,
  )

In [ ]:
tokenized("data")
tokenized("label")

In [ ]:
import sentencepiece as spm
import numpy as np
def bos_eos_padding(dataset,src_max,tgt_max):
  s_en = spm.SentencePieceProcessor(model_file="/content/spm8000_en.model")
  s_zh = spm.SentencePieceProcessor(model_file="/content/spm8000_zh.model")
  padding_src = []
  padding_tgt = []
  len_s = 0
  len_t = 0
  for src,tgt in dataset:
    s = s_en.encode(src, out_type=int)
    s = np.append(s,[3])
    s = np.append([2],np.pad(s,(0, src_max-len(s)), constant_values = 0))
    padding_src.append(s)
    # if len(s)!= len_s:
    #   len_s = len(s)
    #   print(len(s))

    t = s_zh.encode(tgt, out_type=int)
    t = np.append(t,[3])
    t = np.append([2],np.pad(t,(0, tgt_max-len(t)), constant_values = 0))
    padding_tgt.append(t)
    # if len(t)!= len_t:
    #   len_t = len(t)
    #   print(len(t))
  return(list(zip(padding_src,padding_tgt)))

In [ ]:
bos_eos_padding([("hello world","_哈囉")],5,10)

[(array([   2, 7227,  271,    3,    0,    0]),
  array([   2, 2178,    1, 2998, 4387,    3,    0,    0,    0,    0,    0]))]

In [ ]:
s_en = spm.SentencePieceProcessor(model_file="/content/spm8000_en.model")
s_en.encode("hello world!", out_type=int)

[4993, 86, 349]

In [ ]:
import torch.utils.data as data
def data_set_preparing():
    src_set = []
    tgt_set = []

    with open("/content/train_dev/data.txt","r") as in_f :
      for line in in_f:
        src_set.append(line)
    with open("/content/train_dev/label.txt","r") as in_f :
      for line in in_f:
        tgt_set.append(line)

    dataset = list(zip(src_set,tgt_set))
    dataset = bos_eos_padding(dataset,data_info["data"]["max_l"],data_info["label"]["max_l"])
    train_set, valid_set = data.random_split(dataset,[0.9,0.1])
    # print(train_set[0][0])

    with open("/content/train_dev/tokenized_train_data.txt", 'w') as out_f:
      for line_pair in train_set:
        out_f.write(" ".join(str(x) for x in line_pair[0])+"\n")
    with open("/content/train_dev/tokenized_valid_data.txt", 'w') as out_f:
      for line_pair in valid_set:
        out_f.write(" ".join(str(x) for x in line_pair[0])+"\n")
    with open("/content/train_dev/tokenized_train_label.txt", 'w') as out_f:
      for line_pair in train_set:
        out_f.write(" ".join(str(x) for x in line_pair[1])+"\n")
    with open("/content/train_dev/tokenized_valid_label.txt", 'w') as out_f:
      for line_pair in valid_set:
        out_f.write(" ".join(str(x) for x in line_pair[1])+"\n")
data_set_preparing()

In [ ]:
!pip install tqdm

In [ ]:
import torch
from tqdm import tqdm
import numpy as np
from torch.utils.data import Dataset

class myDataset(Dataset):
  padding_id = 0
  def __init__(self,mode):
    if mode == "train":
      self.data_path = "/content/train_dev/tokenized_train_data.txt"
      self.label_path = "/content/train_dev/tokenized_train_label.txt"
    elif mode == "val":
      self.data_path = "/content/train_dev/tokenized_valid_data.txt"
      self.label_path = "/content/train_dev/tokenized_valid_label.txt"

    data = []
    with open(self.data_path,"r") as f :
      d_l = f.readlines()
      for line in tqdm(d_l):
        int_list = [int(i) for i in line.split()]
        data.append(int_list)
    self.data = torch.LongTensor(data)

    label = []
    with open(self.label_path,"r") as f :
      l_l = f.readlines()
      for line in tqdm(l_l):
        int_list = [int(i) for i in line.split()]
        label.append(int_list)
    self.label = torch.LongTensor(np.array(label))

  def __len__(self):
    return len(self.data)

  def __getitem__(self, index):
    return self.data, self.label

  @classmethod
  def padding_mask_batch(cls,batch):
    """Collate a batch of data."""
    src, tgt = zip(*batch)

    src_padding = (src == torch.full(src.size(),cls.padding_id))
    tgt_padding = (tgt == torch.full(tgt.size(),cls.padding_id))

    return src, tgt , src_padding, tgt_padding


In [ ]:
data = []
with open("/content/train_dev/tokenized_train_data.txt","r") as f :
  d_l = f.readlines()
  for line in tqdm(d_l):
    int_list = [int(i) for i in line.split()]
    data.append(int_list)

100%|██████████| 349856/349856 [00:40<00:00, 8713.39it/s] 


In [ ]:
print(len(data),len(data[0]),type(data[0][0]))
data_np = np.array(data)
print(data_np.shape)
data = torch.LongTensor(data)
print(data.size())

349856 402 <class 'int'>
(349856, 402)
torch.Size([349856, 402])


In [ ]:
train_set = myDataset("train",padding_id = 0)
valid_set = myDataset("val",padding_id = 0)

100%|██████████| 38872/38872 [00:01<00:00, 23368.06it/s]


In [ ]:
T = torch.randint(0,3,(3,4,5))
padding = torch.full(T.size(),3)
(T == padding)

tensor([[[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]],

        [[False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False],
         [False, False, False, False, False]]])

In [ ]:
from torch.utils.data import DataLoader

batch_size = 400

train_loader = DataLoader(
  train_set,
  batch_size=batch_size,
  shuffle=True,
  num_workers=8,
  pin_memory=True,
  collate_fn=myDataset.padding_mask_batch
)
valid_loader = DataLoader(
  valid_set,
  batch_size=batch_size,
  num_workers=8,
  pin_memory=True,
  collate_fn=myDataset.padding_mask_batch
)


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
!pip install torchinfo

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchinfo import summary
def attn_mask(input_dim):
      return nn.Transformer.generate_square_subsequent_mask(input_dim)
#attn_output = F.scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)

In [ ]:
class Positional_Encoding(nn.Module):
    def __init__(self,max_sentence_length,embedding_dimension):
      super().__init__()
      self.dropout = nn.Dropout(0.1)
      self.encoding_values = nn.Parameter(nn.init.normal_(torch.empty(max_sentence_length,1, embedding_dimension)))
    def forward(self, x):
        # the shape of x : [batch,length,e_dim]
        # the shape of self.encoding_values : [batch,length,e_dim]
        x = x + self.encoding_values.unsqueeze(0)
        return self.dropout(x)


In [ ]:
class ConformerLayer(nn.Module):
  """
    #------Conformer Encoder Part----------------------------------
    # part 1 feed forward module
    # part 2 multi heads self attention
    # part 3 convolution module
    # part 4 feed forward module
    # part 5 layer norm
    # multi heads self attention module
    #   part 2-1 laryer norm
    #   part 2-2 multi heads self attention
    #   part 2-3 dropout
    # convolution module
    #   part 3-1 layer norm (do this outside the module)
    #   part 3-2 pointwise conv
    #   part 3-3 Glu activation
    #   part 3-4 1D depthwise conv
    #   part 3-5 batch norm
    #   part 3-6 swish activation
    #   part 3-7 pointwise conv
    #   part 3-8 dropout
    """
  def __init__(self,d_model,feedforword,dropout):
    super().__init__()
    self.feed_forward_layer_1 = nn.Sequential(
 #   nn.BatchNorm1d(d_model),
    nn.Linear(d_model, feedforword),
    nn.ReLU(),
    nn.Linear(feedforword, d_model),
 #   nn.BatchNorm1d(d_model),
    )
    self.layer_norm1 = nn.LayerNorm(d_model)
    self.layer_norm2 = nn.LayerNorm(d_model)
    self.attention = torch.nn.MultiheadAttention(d_model, num_heads = 2, dropout=dropout)
    self.DropoutMA = nn.Dropout(dropout)
    self.cnn_module = nn.Sequential(
      # part 3-2
      nn.Conv1d(in_channels = d_model , out_channels = d_model*2 , kernel_size = 1),
      # part 3-3
      nn.GLU(dim = 1),
      # part 3-4
      nn.Conv1d(in_channels = d_model , out_channels = d_model , kernel_size = 3 ,padding = 1, groups = d_model),
      # part 3-5
      nn.BatchNorm1d(d_model),
      # part 3-6
      nn.SiLU(),
      # part 3-7
      nn.Conv1d(in_channels = d_model , out_channels = d_model , kernel_size = 1),
      # part 3-8
      nn.Dropout1d()
    )
    self.feed_forward_layer_2 = nn.Sequential(
 #   nn.BatchNorm1d(d_model),
    nn.Linear(d_model, feedforword),
    nn.ReLU(),
    nn.Linear(feedforword, d_model),
 #   nn.BatchNorm1d(d_model),
    )
    self.layer_norm3 = nn.LayerNorm(d_model)
  def forward(self ,x ):
    # part 1
    x = x + 0.5 * self.feed_forward_layer_1(x)
    # part 2
    x = x.permute(1, 0, 2)
    x = self.layer_norm1(x)
    x = x + self.attention(x,x,x, need_weights=False)[0]
    x = self.DropoutMA(x)
    # part 3
    x = self.layer_norm2(x)
    x = x.permute(1, 2, 0)
    x = x + self.cnn_module(x)
    x = x.permute(0, 2, 1)

    # part 4
    x = x + 0.5 * self.feed_forward_layer_2(x)
    # part 5
    x = self.layer_norm3(x)
    return x

In [ ]:
# class Apply_Padding_Mask(nn.Module):
#   def __init__(self,data,mask_in_booling = None):
#     super().__init__()
#     torch.full
#     self.mask = mask_in_booling
#   def forward(self , x):
#     if self.mask is not None:
#       mask = (torch.zeros_like(self.mask,dtype = float)\
#           .masked_fill_(self.mask, float("-inf")))
#       return data+mask
#     else:
#       return data

In [ ]:
mask = torch.tensor([True,False,True,False])
mask = (
    torch.zeros_like(mask,dtype = float)
    .masked_fill_(mask, float("-inf"))
)
print(mask)

tensor([-inf, 0., -inf, 0.], dtype=torch.float64)


In [ ]:
# from logging import raiseExceptions
from torch.nn.functional import scaled_dot_product_attention
class My_MultiHeadedAttention(nn.Module):
    def __init__(self, embedding_dimension, num_heads, dropout=0.1):
        '''
        embedding_dimension = input dimension
        note that there are residual sublayers in MultiHeadedAttention
        '''
        super().__init__()
        assert embedding_dimension % num_heads == 0, "embed_dim must be divisible by num_heads"

        self.d = embedding_dimension
        self.num_heads = num_heads
        self.linear_for_qkv = nn.Linear(embedding_dimension, 3 * embedding_dimension)
        self.linear_out_project = nn.Linear(embedding_dimension, embedding_dimension)
    def forward(self, input_data , data_padding_mask):
        x = self.linear_for_qkv(input_data)
        # print(x.size())
        query, key, value = x.split(self.d,dim = -1)
        # print(query.size())
        # print(query)
        # query = query.view(x.size(0),x.size(1),self.num_heads,self.d//self.num_heads)
        query,key,value = \
          map(lambda x : x.view(x.size(0),x.size(1),self.num_heads,self.d//self.num_heads),[query,key,value])

        query,key,value = \
          map(lambda x : x.transpose(-2,-3),[query,key,value])

        # print(query.size())
        # print(query)
        x = scaled_dot_product_attention(query,key,value,attn_mask = data_padding_mask, dropout_p =0)
        print(x.size())
        x = x.transpose(-2,-3)
        x = x.view(x.size(0),x.size(1),self.d)
        x = self.linear_out_project(x)
        return x
        # attn_output = attn_output.permute(2, 0, 1, 3).contiguous().view(bsz * tgt_len, embed_dim)
        # attn_output = linear(attn_output, out_proj_weight, out_proj_bias)
        # attn_output = attn_output.view(tgt_len, bsz, attn_output.size(1))

In [ ]:
# model = My_MultiHeadedAttention(6,2)
# input = torch.rand(2,4,6)
# print(input)
# print(model(input))

In [ ]:
import math
class My_Encoder_Layer(nn.Module):
  def __init__(self,max_sentence_length,dictionary_length,embedding_dimension,feedforward_dimension):
    super().__init__()
    self.emb_dim = embedding_dimension
    self.fwd_dim = feedforward_dimension
    # self.apply_encoder_padding_mask = Apply_Padding_Mask()
    self.encoder_embedding = nn.Embedding(dictionary_length,self.emb_dim,padding_idx=0)* math.sqrt(self.emb_dim)
    self.positional_encoding = Positional_Encoding(max_sentence_length,self.emb_dim)

    self.attention = My_MultiHeadedAttention(self.emb_dim, num_heads = 2, dropout=0)
    self.layer_norm_attn = nn.LayerNorm(self.emb_dim)
    self.drop_out_attn_layernorm = nn.Dropout(0)

    self.feedforward = nn.Sequential({
    nn.Linear(self.emb_dim,self.fwd_dim),
    nn.ReLU(),
    nn.Linear(self.fwd_dim,self.emb_dim)
    })
    self.layer_norm_feedforward = nn.LayerNorm(self.emb_dim)
    self.drop_out_feedforward_layernorm = nn.Dropout(0)


  def forward(self,x):
    # x = self.apply_encoder_padding_mask(x)
    x = self.encoder_embedding(x)
    x = self.positional_encoding(x)
    # print(x)
    x = x + self.attention(x)
    x = self.layer_norm_attn(x)
    x = self.drop_out_attn_layernorm(x)

    x = x + self.feedforward(x)
    x = self.layer_norm_feedforward(x)
    x = self.drop_out_feedforward_layernorm(x)

    return x

In [ ]:
model = My_Encoder_Layer(400,8000,128)
input = (torch.randint(0,7999,(32,400,1),dtype = torch.long),torch.randint(0,7999,(32,400,1),dtype = torch.long),myDataset.padding_mask_batch(),myDataset.padding_mask_batch())
#print(summary(model,(32,400,128)))
print(summary(model,input_data = input))
print(model.state_dict().keys())

TypeError: ignored

In [ ]:
class My_Encoder(nn.Module):
  def __init__(self,max_sentence_length,dictionary_length,embedding_dimension,feedforward_dimension,layer_num = 2):
    super().__init__()
    self.encoder = nn.ModuleList([My_Encoder_Layer(max_sentence_length,dictionary_length,\
                    embedding_dimension,feedforward_dimension) for i in range(layer_num)])
  def forward(self,x):
    x = self.encoder(x)
    return x

In [ ]:
import math
class My_Decoder_Layer(nn.Module):
  def __init__(self,max_sentence_length,dictionary_length,embedding_dimension,feedforward_dimension):
    super().__init__()
    self.emb_dim = embedding_dimension
    self.fwd_dim = feedforward_dimension
    self.apply_decoder_padding_mask = Apply_Padding_Mask()
    self.encoder_embedding = nn.Embedding(dictionary_length,self.emb_dim,padding_idx=0)* math.sqrt(self.emb_dim)
    self.positional_encoding = Positional_Encoding(max_sentence_length,self.emb_dim)

    self.attention = My_MultiHeadedAttention(self.emb_dim, num_heads = 2, dropout=0)
    self.layer_norm_attn = nn.LayerNorm(self.emb_dim)
    self.drop_out_attn_layernorm = nn.Dropout(0)

    self.feedforward = nn.Sequential({
    nn.Linear(self.emb_dim,self.fwd_dim),
    nn.ReLU(),
    nn.Linear(self.fwd_dim,self.emb_dim)
    })
    self.layer_norm_feedforward = nn.LayerNorm(self.emb_dim)
    self.drop_out_feedforward_layernorm = nn.Dropout(0)


  def forward(self,x):
    x = self.apply_encoder_padding_mask(x)
    x = self.encoder_embedding(x)
    x = self.positional_encoding(x)
    # print(x)
    x = x + self.attention(x)
    x = self.layer_norm_attn(x)
    x = self.drop_out_attn_layernorm(x)

    x = x + self.feedforward(x)
    x = self.layer_norm_feedforward(x)
    x = self.drop_out_feedforward_layernorm(x)

    return x

In [ ]:
# from torchinfo import summary
# model = nn.TransformerDecoderLayer(d_model=40, nhead=1)
# print(summary(model,((32,128,40),(32,128,40)),tgt_mask = attn_mask(32),tgt_key_padding_mask = padding_mask))

In [ ]:
class LabelSmoothing(nn.Module):
    """Implement label smoothing."""

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False)
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, Variable(true_dist, requires_grad=False))


class Embeddings(nn.Module):

class PositionalEncoding(nn.Module):

def attention(query, key, value, mask=None, dropout=None):

class MultiHeadedAttention(nn.Module):

class LayerNorm(nn.Module):

class SublayerConnection(nn.Module):

def clones(module, N):

class PositionwiseFeedForward(nn.Module):

class Encoder(nn.Module):

class EncoderLayer(nn.Module):

class Decoder(nn.Module):
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        # 复制N个encoder layer
        self.layers = clones(layer, N)
        # Layer Norm
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        """
        使用循环连续decode N次(这里为6次)
        这里的Decoderlayer会接收一个对于输入的attention mask处理
        和一个对输出的attention mask + subsequent mask处理
        """
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


class DecoderLayer(nn.Module):
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        # Self-Attention
        self.self_attn = self_attn
        # 与Encoder传入的Context进行Attention
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        # 用m来存放encoder的最终hidden表示结果
        m = memory

        # Self-Attention：注意self-attention的q，k和v均为decoder hidden
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        # Context-Attention：注意context-attention的q为decoder hidden，而k和v为encoder hidden
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


class Transformer(nn.Module):
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed
        self.tgt_embed = tgt_embed
        self.generator = generator

    def encode(self, src, src_mask):
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

    def forward(self, src, tgt, src_mask, tgt_mask):
        # encoder的结果作为decoder的memory参数传入，进行decode
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)


class Generator(nn.Module):
    # vocab: tgt_vocab
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        # decode后的结果，先进入一个全连接层变为词典大小的向量
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        # 然后再进行log_softmax操作(在softmax结果上再做多一次log运算)
        return F.log_softmax(self.proj(x), dim=-1)


def make_model(src_vocab, tgt_vocab, N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
    c = copy.deepcopy
    # 实例化Attention对象
    attn = MultiHeadedAttention(h, d_model).to(DEVICE)
    # 实例化FeedForward对象
    ff = PositionwiseFeedForward(d_model, d_ff, dropout).to(DEVICE)
    # 实例化PositionalEncoding对象
    position = PositionalEncoding(d_model, dropout).to(DEVICE)
    # 实例化Transformer模型对象
    model = Transformer(
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout).to(DEVICE), N).to(DEVICE),
        Decoder(DecoderLayer(d_model, c(attn), c(attn), c(ff), dropout).to(DEVICE), N).to(DEVICE),
        nn.Sequential(Embeddings(d_model, src_vocab).to(DEVICE), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab).to(DEVICE), c(position)),
        Generator(d_model, tgt_vocab)).to(DEVICE)

    # This was important from their code.
    # Initialize parameters with Glorot / fan_avg.
    for p in model.parameters():
        if p.dim() > 1:
            # 这里初始化采用的是nn.init.xavier_uniform
            nn.init.xavier_uniform_(p)
    return model.to(DEVICE)


def batch_greedy_decode(model, src, src_mask, max_len=64, start_symbol=2, end_symbol=3):
    batch_size, src_seq_len = src.size()
    results = [[] for _ in range(batch_size)]
    stop_flag = [False for _ in range(batch_size)]
    count = 0

    memory = model.encode(src, src_mask)
    tgt = torch.Tensor(batch_size, 1).fill_(start_symbol).type_as(src.data)

    for s in range(max_len):
        tgt_mask = subsequent_mask(tgt.size(1)).expand(batch_size, -1, -1).type_as(src.data)
        out = model.decode(memory, src_mask, Variable(tgt), Variable(tgt_mask))

        prob = model.generator(out[:, -1, :])
        pred = torch.argmax(prob, dim=-1)

        tgt = torch.cat((tgt, pred.unsqueeze(1)), dim=1)
        pred = pred.cpu().numpy()
        for i in range(batch_size):
            # print(stop_flag[i])
            if stop_flag[i] is False:
                if pred[i] == end_symbol:
                    count += 1
                    stop_flag[i] = True
                else:
                    results[i].append(pred[i].item())
            if count == batch_size:
                break

    return results


def greedy_decode(model, src, src_mask, max_len=64, start_symbol=2, end_symbol=3):
    """传入一个训练好的模型，对指定数据进行预测"""
    # 先用encoder进行encode
    memory = model.encode(src, src_mask)
    # 初始化预测内容为1×1的tensor，填入开始符('BOS')的id，并将type设置为输入数据类型(LongTensor)
    ys = torch.ones(1, 1).fill_(start_symbol).type_as(src.data)
    # 遍历输出的长度下标
    for i in range(max_len - 1):
        # decode得到隐层表示
        out = model.decode(memory,
                           src_mask,
                           Variable(ys),
                           Variable(subsequent_mask(ys.size(1)).type_as(src.data)))
        # 将隐藏表示转为对词典各词的log_softmax概率分布表示
        prob = model.generator(out[:, -1])
        # 获取当前位置最大概率的预测词id
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        if next_word == end_symbol:
            break
        # 将当前位置预测的字符id与之前的预测内容拼接起来
        ys = torch.cat([ys,
                        torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=1)
    return ys

134023434658160 134023434658160
<function StaticMethod.set_string at 0x79e4c336e170> <function StaticMethod.set_string at 0x79e4c336e170>
b b
